In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

if "../../" not in sys.path:
    sys.path.append("../../")
    print("[sys.path]:", sys.path)

In [ ]:
from src.data.babel import BabelDataset

In [ ]:
# NOTE: parameters
pipeline_name = None
deciles = [1, 2, 2.5, 3, 4, 5, 6, 7, 7.5, 8, 9, 9.5, 9.99]

In [ ]:
from src.data.pipelines._registery import PIPELINE_REGISTRY

pipeline_names = [name for names in PIPELINE_REGISTRY.values() for name in names]

if pipeline_name not in pipeline_names:
    raise ValueError(f"Pipeline '{pipeline_name}' is not registered. Available pipelines: {pipeline_names}")
else:
    print(f"[validation]: pipeline '{pipeline_name}' is registered and available.")

In [ ]:
train_dataset = BabelDataset(
    split="train",
    pipeline=pipeline_name
)
print("[train_dataset.fingerprint]:", train_dataset.fingerprint)
validation_dataset = BabelDataset(
    split="validation",
    pipeline=pipeline_name
)
print("[validation_dataset.fingerprint]:", validation_dataset.fingerprint)

In [ ]:
print("[#train_dataset]:", len(train_dataset))
print("[#validation_dataset]:", len(validation_dataset))

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter, defaultdict

from src.constants import (
    DEFAULT_FPS
)

In [ ]:
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
def analyze_prompts(dataset, dataset_name="Dataset"):
    all_prompts = []
    sample_prompt_counts = []
    source_counts = Counter()
    is_sequence_counts = Counter()
    durations = []
    sequence_durations = []
    frame_durations = []
    spans_per_prompt = defaultdict(list)
    
    motion_lengths_frames = []
    motion_lengths_seconds = []
    spans_per_motion = []
    spans_durations_per_motion = []
    
    for i, sample in enumerate(dataset):
        prompts_list = sample.get("prompts", [])
        sample_prompt_counts.append(len(prompts_list))
        
        motion = sample.get("motion", {})
        if isinstance(motion, dict) and "new_joint_vecs" in motion:
            motion_length = len(motion["new_joint_vecs"])
            motion_lengths_frames.append(motion_length)
            motion_lengths_seconds.append(motion_length / DEFAULT_FPS)
        
        motion_spans = []
        motion_spans_durations = []
        
        for prompt_data in prompts_list:
            prompt_text = prompt_data.get("text", "")
            span = prompt_data.get("span", [])
            source = prompt_data.get("source", "unknown")
            is_sequence = prompt_data.get("is_sequence", True)
            
            all_prompts.append(prompt_text)
            source_counts[source] += 1
            is_sequence_counts[is_sequence] += 1
            
            if len(span) == 2:
                duration_frames = span[1] - span[0]
                durations.append(duration_frames)
                spans_per_prompt[prompt_text].append(span)
                motion_spans.append(span)
                motion_spans_durations.append(duration_frames)
                
                if is_sequence:
                    sequence_durations.append(duration_frames)
                else:
                    frame_durations.append(duration_frames)
        
        spans_per_motion.append(len(motion_spans))
        spans_durations_per_motion.append(sum(motion_spans_durations) if motion_spans_durations else 0)
    
    return {
        'all_prompts': all_prompts,
        'sample_prompt_counts': sample_prompt_counts,
        'source_counts': source_counts,
        'is_sequence_counts': is_sequence_counts,
        'durations': durations,
        'sequence_durations': sequence_durations,
        'frame_durations': frame_durations,
        'spans_per_prompt': spans_per_prompt,
        'dataset_name': dataset_name,
        'motion_lengths_frames': motion_lengths_frames,
        'motion_lengths_seconds': motion_lengths_seconds,
        'spans_per_motion': spans_per_motion,
        'spans_durations_per_motion': spans_durations_per_motion
    }

In [ ]:
import pickle
import os

def get_analysis_with_cache(dataset, dataset_name, cache_dir="./cache.local"):
    os.makedirs(cache_dir, exist_ok=True)
    fingerprint = getattr(dataset, 'fingerprint', None)
    
    cache_path = os.path.join(cache_dir, f"{pipeline_name}_{dataset_name.lower()}_{fingerprint}.pkl")
    
    if fingerprint and os.path.exists(cache_path):
        print(f"[cache]: Loading {dataset_name} analysis from {cache_path}")
        with open(cache_path, "rb") as f:
            return pickle.load(f)
    else:
        print(f"[cache]: Computing {dataset_name} analysis and saving to {cache_path}")
        analysis = analyze_prompts(dataset, dataset_name)
        if fingerprint:
            with open(cache_path, "wb") as f:
                pickle.dump(analysis, f)
        return analysis

train_analysis = get_analysis_with_cache(train_dataset, "Train")
validation_analysis = get_analysis_with_cache(validation_dataset, "Validation")

In [ ]:
print("=== BASIC STATISTICS ===")
print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Validation Dataset: {len(validation_dataset)} samples")
print()

for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    print(f"{name} Dataset:")
    print(f"  Total prompts: {len(analysis['all_prompts'])}")
    print(f"  Unique prompts: {len(set(analysis['all_prompts']))}")
    print(f"  Avg prompts per sample: {np.mean(analysis['sample_prompt_counts']):.2f}")
    print(f"  Min prompts per sample: {np.min(analysis['sample_prompt_counts'])}")
    print(f"  Max prompts per sample: {np.max(analysis['sample_prompt_counts'])}")
    print()
    
    print(f"  Source distribution:")
    for source, count in analysis['source_counts'].items():
        print(f"    {source}: {count} ({count/len(analysis['all_prompts'])*100:.1f}%)")
    print()
    
    print(f"  Sequence vs Frame annotations:")
    for is_seq, count in analysis['is_sequence_counts'].items():
        label = "Sequence" if is_seq else "Frame"
        print(f"    {label}: {count} ({count/len(analysis['all_prompts'])*100:.1f}%)")
    print()
    
    if analysis['durations']:
        print(f"  Duration statistics (frames):")
        print(f"    Min: {np.min(analysis['durations'])} frames ({np.min(analysis['durations'])/DEFAULT_FPS:.2f}s)")
        print(f"    Mean: {np.mean(analysis['durations']):.2f} frames ({np.mean(analysis['durations'])/DEFAULT_FPS:.2f}s)")
        print(f"    Max: {np.max(analysis['durations'])} frames ({np.max(analysis['durations'])/DEFAULT_FPS:.2f}s)")
        print(f"    Std: {np.std(analysis['durations']):.2f} frames ({np.std(analysis['durations'])/DEFAULT_FPS:.2f}s)")
    print("-" * 50)

<div class="alert alert-info">

#### **Number of Motions**

This shows the total number of motion sequences in each dataset split.

</div>

In [ ]:
print("=== NUMBER OF MOTIONS ===")
for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    num_motions = len(analysis['motion_lengths_frames'])
    print(f"{name} Dataset: {num_motions} motions")

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
datasets = ['Train', 'Validation']
motion_counts = [len(train_analysis['motion_lengths_frames']), len(validation_analysis['motion_lengths_frames'])]
colors = ['C0', 'C1']

bars = ax.bar(datasets, motion_counts, color=colors, alpha=0.7)
ax.set_title('Number of Motions per Dataset')
ax.set_ylabel('Number of Motions')
ax.grid(True, alpha=0.3)

for bar, count in zip(bars, motion_counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(motion_counts)*0.01, 
            str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

<div class="alert alert-info">

#### **Motion Length Distribution**

This shows the distribution of motion sequence lengths in both frames and seconds, with horizontal lines indicating minimum, maximum, and average values.

</div>

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Motion Length Distribution', fontsize=16)

for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    lengths_frames = analysis['motion_lengths_frames']
    lengths_seconds = analysis['motion_lengths_seconds']
    
    if lengths_frames:
        # NOTE: frames distribution
        axes[i, 0].hist(lengths_frames, bins=30, alpha=0.7, color=f'C{i}')
        axes[i, 0].set_title(f'{name} - Motion Length (Frames)')
        axes[i, 0].set_xlabel('Length (frames)')
        axes[i, 0].set_ylabel('Frequency')
        axes[i, 0].grid(True, alpha=0.3)
        
        min_frames = np.min(lengths_frames)
        max_frames = np.max(lengths_frames)
        avg_frames = np.mean(lengths_frames)
        
        axes[i, 0].axvline(min_frames, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_frames}')
        axes[i, 0].axvline(max_frames, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_frames}')
        axes[i, 0].axvline(avg_frames, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_frames:.1f}')
        
        for d in deciles:
            decile = np.percentile(lengths_frames, d*10)
            # axes[i, 0].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}')
            axes[i, 0].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(lengths_frames)*d/10):,}'
            )
        axes[i, 0].legend()
        
        # NOTE: seconds distribution
        axes[i, 1].hist(lengths_seconds, bins=30, alpha=0.7, color=f'C{i}')
        axes[i, 1].set_title(f'{name} - Motion Length (Seconds)')
        axes[i, 1].set_xlabel('Length (seconds)')
        axes[i, 1].set_ylabel('Frequency')
        axes[i, 1].grid(True, alpha=0.3)
        
        min_seconds = np.min(lengths_seconds)
        max_seconds = np.max(lengths_seconds)
        avg_seconds = np.mean(lengths_seconds)
        
        axes[i, 1].axvline(min_seconds, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_seconds:.1f}s')
        axes[i, 1].axvline(max_seconds, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_seconds:.1f}s')
        axes[i, 1].axvline(avg_seconds, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_seconds:.1f}s')
        
        for d in deciles:
            decile = np.percentile(lengths_seconds, d*10)
            # axes[i, 1].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}s')
            axes[i, 1].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(lengths_seconds)*d/10):,}'
            )
        axes[i, 1].legend()

plt.tight_layout()
plt.show()

print("=== MOTION LENGTH STATISTICS ===")
for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    lengths_frames = analysis['motion_lengths_frames']
    lengths_seconds = analysis['motion_lengths_seconds']
    
    if lengths_frames:
        print(f"\n{name} Dataset:")
        print(f"  Frames - Min: {np.min(lengths_frames)}, Mean: {np.mean(lengths_frames):.2f}, Max: {np.max(lengths_frames)}")
        print(f"  Seconds - Min: {np.min(lengths_seconds):.2f}, Mean: {np.mean(lengths_seconds):.2f}, Max: {np.max(lengths_seconds):.2f}")
        print(f"  75th percentile - Frames: {np.percentile(lengths_frames, 75):.2f}, Seconds: {np.percentile(lengths_seconds, 75):.2f}")
    print("-" * 50)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Motion Length Distribution - Box Plots', fontsize=16)

datasets = ['Train', 'Validation']
for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    lengths_frames = analysis['motion_lengths_frames']
    lengths_seconds = analysis['motion_lengths_seconds']
    
    if lengths_frames:
        box_data_frames = [lengths_frames]
        bp1 = axes[0].boxplot(box_data_frames, positions=[i+1], widths=0.6, patch_artist=True)
        bp1['boxes'][0].set_facecolor(f'C{i}')
        bp1['boxes'][0].set_alpha(0.7)
        
        box_data_seconds = [lengths_seconds]
        bp2 = axes[1].boxplot(box_data_seconds, positions=[i+1], widths=0.6, patch_artist=True)
        bp2['boxes'][0].set_facecolor(f'C{i}')
        bp2['boxes'][0].set_alpha(0.7)

axes[0].set_title('Motion Length Distribution - Frames')
axes[0].set_ylabel('Length (frames)')
axes[0].set_xticks([1, 2])
axes[0].set_xticklabels(datasets)
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Motion Length Distribution - Seconds')
axes[1].set_ylabel('Length (seconds)')
axes[1].set_xticks([1, 2])
axes[1].set_xticklabels(datasets)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<div class="alert alert-info">

#### **Prompts per Motion Distribution**

This shows how many prompts are associated with each motion sequence, with horizontal lines indicating minimum, maximum, and average values.

</div>

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Prompts per Motion Distribution', fontsize=16)

for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    prompts_per_motion = analysis['sample_prompt_counts']
    
    if prompts_per_motion:
        axes[i].hist(prompts_per_motion, bins=30, alpha=0.7, color=f'C{i}')
        axes[i].set_title(f'{name} - Prompts per Motion')
        axes[i].set_xlabel('Number of Prompts')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        min_prompts = np.min(prompts_per_motion)
        max_prompts = np.max(prompts_per_motion)
        avg_prompts = np.mean(prompts_per_motion)
        
        axes[i].axvline(min_prompts, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_prompts}')
        axes[i].axvline(max_prompts, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_prompts}')
        axes[i].axvline(avg_prompts, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_prompts:.1f}')
        
        for d in deciles:
            decile = np.percentile(prompts_per_motion, d*10)
            # axes[i].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}')
            axes[i].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(prompts_per_motion)*d/10):,}'
            )
        axes[i].legend()

plt.tight_layout()
plt.show()

print("=== PROMPTS PER MOTION STATISTICS ===")
for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    prompts_per_motion = analysis['sample_prompt_counts']
    
    if prompts_per_motion:
        print(f"\n{name} Dataset:")
        print(f"  Min: {np.min(prompts_per_motion)}, Mean: {np.mean(prompts_per_motion):.2f}, Max: {np.max(prompts_per_motion)}")
        print(f"  75th percentile: {np.percentile(prompts_per_motion, 75):.2f}")
    print("-" * 50)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Prompts per Motion Distribution - Box Plot', fontsize=16)

datasets = ['Train', 'Validation']
box_data = []
colors = ['C0', 'C1']

for i, analysis in enumerate([train_analysis, validation_analysis]):
    prompts_per_motion = analysis['sample_prompt_counts']
    if prompts_per_motion:
        box_data.append(prompts_per_motion)

bp = ax.boxplot(box_data, positions=[1, 2], widths=0.6, patch_artist=True)
for i, box in enumerate(bp['boxes']):
    box.set_facecolor(colors[i])
    box.set_alpha(0.7)

ax.set_title('Prompts per Motion Distribution')
ax.set_ylabel('Number of Prompts')
ax.set_xticks([1, 2])
ax.set_xticklabels(datasets)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<div class="alert alert-info">

#### **Unique Prompts per Motion Distribution**

This shows the distribution of the number of unique prompts per motion sequence.

</div>

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Unique Prompts per Motion Distribution', fontsize=16)

for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    all_prompts = analysis['all_prompts']
    sample_prompt_counts = analysis['sample_prompt_counts']
    
    unique_counts = []
    idx = 0
    for count in sample_prompt_counts:
        prompts = all_prompts[idx:idx+count]
        unique_counts.append(len(set(prompts)))
        idx += count

    if unique_counts:
        axes[i].hist(unique_counts, bins=30, alpha=0.7, color=f'C{i}')
        axes[i].set_title(f'{name} - Unique Prompts per Motion')
        axes[i].set_xlabel('Number of Unique Prompts')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)

        min_val = np.min(unique_counts)
        max_val = np.max(unique_counts)
        avg_val = np.mean(unique_counts)

        axes[i].axvline(min_val, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_val}')
        axes[i].axvline(max_val, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_val}')
        axes[i].axvline(avg_val, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_val:.1f}')
        
        for d in deciles:
            decile = np.percentile(unique_counts, d*10)
            # axes[i].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}')
            axes[i].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(unique_counts)*d/10):,}'
            )
            
        axes[i].legend()

plt.tight_layout()
plt.show()

<div class="alert alert-info">

#### **Spans per Motion Distribution**

This shows how many spans (temporal segments) are associated with each motion sequence, with horizontal lines indicating minimum, maximum, and average values.

</div>

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Spans per Motion Distribution', fontsize=16)

for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    spans_per_motion = analysis['spans_per_motion']
    
    if spans_per_motion:
        axes[i].hist(spans_per_motion, bins=30, alpha=0.7, color=f'C{i}')
        axes[i].set_title(f'{name} - Spans per Motion')
        axes[i].set_xlabel('Number of Spans')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        min_spans = np.min(spans_per_motion)
        max_spans = np.max(spans_per_motion)
        avg_spans = np.mean(spans_per_motion)
        
        axes[i].axvline(min_spans, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_spans}')
        axes[i].axvline(max_spans, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_spans}')
        axes[i].axvline(avg_spans, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_spans:.1f}')
        
        for d in deciles:
            decile = np.percentile(spans_per_motion, d*10)
            # axes[i].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}')
            axes[i].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(spans_per_motion)*d/10):,}'
            )
        axes[i].legend()

plt.tight_layout()
plt.show()

print("=== SPANS PER MOTION STATISTICS ===")
for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    spans_per_motion = analysis['spans_per_motion']
    
    if spans_per_motion:
        print(f"\n{name} Dataset:")
        print(f"  Min: {np.min(spans_per_motion)}, Mean: {np.mean(spans_per_motion):.2f}, Max: {np.max(spans_per_motion)}")
        print(f"  75th percentile: {np.percentile(spans_per_motion, 75):.2f}")
    print("-" * 50)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Spans per Motion Distribution - Box Plot', fontsize=16)

datasets = ['Train', 'Validation']
box_data = []
colors = ['C0', 'C1']

for i, analysis in enumerate([train_analysis, validation_analysis]):
    spans_per_motion = analysis['spans_per_motion']
    if spans_per_motion:
        box_data.append(spans_per_motion)

bp = ax.boxplot(box_data, positions=[1, 2], widths=0.6, patch_artist=True)
for i, box in enumerate(bp['boxes']):
    box.set_facecolor(colors[i])
    box.set_alpha(0.7)

ax.set_title('Spans per Motion Distribution')
ax.set_ylabel('Number of Spans')
ax.set_xticks([1, 2])
ax.set_xticklabels(datasets)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<div class="alert alert-info">

#### **Spans Duration Distribution**

This shows the distribution of individual span durations in frames, with horizontal lines indicating minimum, maximum, and average values.

</div>

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Spans Duration Distribution', fontsize=16)

for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    span_durations = analysis['durations']
    
    if span_durations:
        axes[i].hist(span_durations, bins=30, alpha=0.7, color=f'C{i}')
        axes[i].set_title(f'{name} - Span Duration (Frames)')
        axes[i].set_xlabel('Duration (frames)')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        min_duration = np.min(span_durations)
        max_duration = np.max(span_durations)
        avg_duration = np.mean(span_durations)
        
        axes[i].axvline(min_duration, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_duration}')
        axes[i].axvline(max_duration, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_duration}')
        axes[i].axvline(avg_duration, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_duration:.1f}')
        
        for d in deciles:
            decile = np.percentile(span_durations, d*10)
            # axes[i].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}')
            axes[i].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(span_durations)*d/10):,}'
            )
        axes[i].legend()

plt.tight_layout()
plt.show()

print("=== SPANS DURATION STATISTICS ===")
for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    span_durations = analysis['durations']
    
    if span_durations:
        print(f"\n{name} Dataset:")
        print(f"  Min: {np.min(span_durations)} frames ({np.min(span_durations)/DEFAULT_FPS:.2f}s)")
        print(f"  Mean: {np.mean(span_durations):.2f} frames ({np.mean(span_durations)/DEFAULT_FPS:.2f}s)")
        print(f"  Max: {np.max(span_durations)} frames ({np.max(span_durations)/DEFAULT_FPS:.2f}s)")
        print(f"  75th percentile: {np.percentile(span_durations, 75):.2f} frames ({np.percentile(span_durations, 75)/DEFAULT_FPS:.2f}s)")
    print("-" * 50)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Spans Duration Distribution - Box Plot', fontsize=16)

datasets = ['Train', 'Validation']
box_data = []
colors = ['C0', 'C1']

for i, analysis in enumerate([train_analysis, validation_analysis]):
    span_durations = analysis['durations']
    if span_durations:
        span_durations_seconds = [d / DEFAULT_FPS for d in span_durations]
        box_data.append(span_durations_seconds)

bp = ax.boxplot(box_data, positions=[1, 2], widths=0.6, patch_artist=True)
for i, box in enumerate(bp['boxes']):
    box.set_facecolor(colors[i])
    box.set_alpha(0.7)

ax.set_title('Spans Duration Distribution')
ax.set_ylabel('Duration (seconds)')
ax.set_xticks([1, 2])
ax.set_xticklabels(datasets)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<div class="alert alert-info">

#### **Total Spans per Motion (Per Motion Analysis)**

This shows the total span duration per motion sequence, indicating how much of each motion is covered by annotated spans.

</div>

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Total Spans Duration per Motion', fontsize=16)

for i, analysis in enumerate([train_analysis, validation_analysis]):
    name = analysis['dataset_name']
    spans_durations_per_motion = analysis['spans_durations_per_motion']
    
    if spans_durations_per_motion:
        axes[i].hist(spans_durations_per_motion, bins=30, alpha=0.7, color=f'C{i}')
        axes[i].set_title(f'{name} - Total Spans Duration per Motion')
        axes[i].set_xlabel('Total Duration (frames)')
        axes[i].set_ylabel('Number of Motions')
        axes[i].grid(True, alpha=0.3)
        
        min_duration = np.min(spans_durations_per_motion)
        max_duration = np.max(spans_durations_per_motion)
        avg_duration = np.mean(spans_durations_per_motion)
        
        axes[i].axvline(min_duration, color='red', linestyle='--', alpha=0.7, label=f'Min: {min_duration}')
        axes[i].axvline(max_duration, color='orange', linestyle='--', alpha=0.7, label=f'Max: {max_duration}')
        axes[i].axvline(avg_duration, color='green', linestyle='--', alpha=0.7, label=f'Avg: {avg_duration:.1f}')
        
        for d in deciles:
            decile = np.percentile(spans_durations_per_motion, d*10)
            # axes[i].axvline(decile, color='gray', linestyle='-.', alpha=0.5, label=f'D{d}: {decile:.1f}')
            axes[i].axvline(
                decile,
                color='gray',
                linestyle='-.',
                alpha=0.5,
                label=f'D{d}: {decile:.1f} ({d*10:.0f}%) - N={int(len(spans_durations_per_motion)*d/10):,}'
            )
        axes[i].legend()

plt.tight_layout()
plt.show()

print("=== TOTAL SPANS DURATION PER MOTION STATISTICS ===")
for analysis in [train_analysis, validation_analysis]:
    name = analysis['dataset_name']
    spans_durations_per_motion = analysis['spans_durations_per_motion']
    
    if spans_durations_per_motion:
        print(f"\n{name} Dataset:")
        print(f"  Min: {np.min(spans_durations_per_motion)} frames ({np.min(spans_durations_per_motion)/DEFAULT_FPS:.2f}s)")
        print(f"  Mean: {np.mean(spans_durations_per_motion):.2f} frames ({np.mean(spans_durations_per_motion)/DEFAULT_FPS:.2f}s)")
        print(f"  Max: {np.max(spans_durations_per_motion)} frames ({np.max(spans_durations_per_motion)/DEFAULT_FPS:.2f}s)")
        print(f"  75th percentile: {np.percentile(spans_durations_per_motion, 75):.2f} frames ({np.percentile(spans_durations_per_motion, 75)/DEFAULT_FPS:.2f}s)")
    print("-" * 50)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Total Spans Duration per Motion Distribution - Box Plot', fontsize=16)

datasets = ['Train', 'Validation']
box_data = []
colors = ['C0', 'C1']

for i, analysis in enumerate([train_analysis, validation_analysis]):
    spans_durations_per_motion = analysis['spans_durations_per_motion']
    if spans_durations_per_motion:
        spans_durations_seconds = [d / DEFAULT_FPS for d in spans_durations_per_motion]
        box_data.append(spans_durations_seconds)

bp = ax.boxplot(box_data, positions=[1, 2], widths=0.6, patch_artist=True)
for i, box in enumerate(bp['boxes']):
    box.set_facecolor(colors[i])
    box.set_alpha(0.7)

ax.set_title('Total Spans Duration per Motion Distribution')
ax.set_ylabel('Total Duration (seconds)')
ax.set_xticks([1, 2])
ax.set_xticklabels(datasets)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()